In [ ]:
import sqlite3

conn = sqlite3.connect("Data/users.db")  # in-memory DB
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    username TEXT,
    password TEXT
)
""")

cursor.execute("INSERT INTO users VALUES (1, 'admin', 'admin123')")
cursor.execute("INSERT INTO users VALUES (2, 'user', 'user123')")

conn.commit()

SQL Injection

In [5]:
def insecure_login(username):
    query = f"SELECT * FROM users WHERE username = '{username}'"
    print("Executing:", query)
    cursor.execute(query)
    return cursor.fetchall()

# Attacker input
result = insecure_login("' OR 1=1 --")
result

Executing: SELECT * FROM users WHERE username = '' OR 1=1 --'


[(1, 'admin', 'admin123'), (2, 'user', 'user123')]

Parameterized Queries

In [6]:
def secure_login(username):
    query = "SELECT * FROM users WHERE username = ?"
    cursor.execute(query, (username,))
    return cursor.fetchall()

secure_login("' OR 1=1 --")

[]

FILE HANDLING

In [17]:
# Directory Traversal Example
def read_file(filename):
    with open(filename, "r") as f:
        return f.read()

read_file("Data/etc/passwd") 


'password1\npassword2'

In [33]:
import os


# Insecure function 

def read_file(filename):
    with open(filename, "r") as f:
        return f.read()

# Secure function (safe handling)

BASE_DIR = "uploads"

def secure_read_file(filename):
    safe_name = os.path.basename(filename)   # strips directory traversal
    full_path = os.path.join(BASE_DIR, safe_name)

    # Ensure path stays inside uploads
    if not full_path.startswith(BASE_DIR):
        raise Exception("Invalid file path")

    with open(full_path, "r") as f:
        return f.read()


#insecure file handling
print(read_file("C:\\temp\\secret.txt"))   

#secure file handling
try:
    # This will try to sanitize and force into uploads
    print(secure_read_file("C:\\temp\\secret.txt"))  # Attack blocked
except Exception as e:
    print("Blocked:", e)
except FileNotFoundError:
    print("File not found in uploads (safe behavior)")

ÿþT O P 
 
 S E C R E T 
 
 D A T A 
 
 
Blocked: [Errno 2] No such file or directory: 'uploads\\secret.txt'
